In [ ]:
import dynamiqs as dq
import scipy as sp
import jax
import jax.numpy as jnp # the JAX version of numpy
from matplotlib import pyplot as plt

In [ ]:
jax.devices()
dq.set_device('cpu')

dq.set_precision('simple')

In [3]:
na = 20
nb = 5
ω_a0 = 2*jnp.pi*5.26e9
ω_b0 = 2*jnp.pi*7.70e9
 
φ_a = 0.06
φ_b = 0.29

E_J = 42.76e9* sp.h
 
ΔE_J = 0.47e9 * sp.h


ω_d = 2*jnp.pi*7.623e9
ω_p = 2*jnp.pi*2.891e9
 
ϵ_d = -2*jnp.pi*3.815e6
ϵ_p = 0.122

a,b = dq.destroy(na,nb)

H_0 = ω_a0* dq.dag(a)@a + ω_b0* dq.dag(b)@b 

H_ATS = -2* E_J*(jnp.sin(ϵ_p)*sin(φ_a))

SyntaxError: invalid character '−' (U+2212) (3554055583.py, line 31)